In [1]:
import pickle
import pandas as pd
import csv
import json
import os

### Evaluate how much was changed in the Review process of the Crowdsourced Annotations

In [2]:
#Load Reviewed Dataset
with open('reviewed_data/initial_1300.json') as json_file:
    rewiewed_json = json.load(json_file)
    
review_dict = {}
for row in rewiewed_json:
    review_dict[row['data']['dream_id']] = {'completions':row['completions'], 'data':row['data'] }

In [3]:
#Load original exports
labelstudio_export_list = ['appen_results/label_studio_Aggregated_Report_1726283.json',
'appen_results/label_studio_Aggregated_Report_1726283_2.json',
'appen_results/label_studio_Aggregated_Report_1726283_3.json',
'appen_results/label_studio_Aggregated_Report_1734122.json'
                      ]
export_dict = {}

for file_path in labelstudio_export_list:
    
    with open(file_path) as json_file:
        export = json.load(json_file)
    
    for row in export:
        export_dict[row['data']['dream_id']] = {'completions':row['completions'], 'data':row['data'] }

    


In [4]:






len(export_dict)

880

In [5]:
contained = 0
not_contained = 0
for export_key in export_dict.keys():
    if export_key in review_dict:
        contained+=1
    else:
        not_contained +=1
print(contained, ' elements are in the review dataset and ', not_contained, ' elements are not.')

880  elements are in the review dataset and  0  elements are not.


In [6]:
#Evaluate How much of the reviewed annotations was already annotated in the crowdsourcing process                    
#Characterwise                        
total_overlap = 0
total_not_overlap = 0
whole_row_missing=0

for rev_key in review_dict.keys():
    if rev_key in export_dict:
        export_comp = export_dict[rev_key]['completions']
        review_comp = review_dict[rev_key]['completions']
        
        #Loop over all reviewed annotations
        for rev_tag in review_comp[0]['result']:
            r_span = range(rev_tag['value']['start'], rev_tag['value']['end'] - 1)
            r_spanset = set(r_span)
            tag_overlap = 0
            tag_not_overlap=0
            
            if rev_tag['value']['labels'][0] == 'Drug':#only check drug annotations
                #check each annotation in original export if it overlaps..
                for exp_tag in export_comp[0]['result']:
                    exp_span = range(exp_tag['value']['start'], exp_tag['value']['end'] - 1)
                    tag_overlap += len(r_spanset.intersection(exp_span))
                
                #Calculate what was not found in the original annotation and store mikro results
                tag_not_overlap = len(r_span) - tag_overlap
                total_overlap += tag_overlap
                total_not_overlap += tag_not_overlap
                if tag_not_overlap < 0:
                    print('ERROR')
                    
    else:
        whole_row_missing+=1

total_chars = total_overlap + total_not_overlap
print('Of ', total_chars, ' characters included in tags after the review ', total_overlap, 
      ' were already present as is in the export')
print('This is ', total_overlap/total_chars, '%')
print('We are missing ', whole_row_missing, ' rows where I cannot find the original export')
       

Of  18395  characters included in tags after the review  13647  were already present as is in the export
This is  0.7418863821690677 %
We are missing  138  rows where I cannot find the original export


In [7]:
#Evaluate How much of the original crowdsourcing annotations were found in the reviewed dataset.   
#Characterwise

total_overlap = 0
total_not_overlap = 0
whole_row_missing=0

for export_key in export_dict.keys():
    if export_key in review_dict:
        export_comp = export_dict[export_key]['completions']
        review_comp = review_dict[export_key]['completions']
        
        #Loop over all reviewed annotations
        for exp_tag in export_comp[0]['result']:
            e_span = range(exp_tag['value']['start'], exp_tag['value']['end'] - 1)
            e_spanset = set(e_span)
            tag_overlap = 0
            tag_not_overlap=0
            
            if exp_tag['value']['labels'][0] == 'Drug':#only check drug annotations
                #check each annotation in original export if it overlaps..
                for rev_tag in review_comp[0]['result']:
                    r_span = range(rev_tag['value']['start'], rev_tag['value']['end'] - 1)
                    tag_overlap += len(e_spanset.intersection(r_span))
                
                #Calculate what was not found in the original annotation and store mikro results
                tag_not_overlap = len(e_span) - tag_overlap
                total_overlap += tag_overlap
                total_not_overlap += tag_not_overlap
                if tag_not_overlap < 0:
                    print('ERROR')
                    
    else:
        whole_row_missing+=1

total_chars = total_overlap + total_not_overlap
print('Of ', total_chars, ' characters orginally annotated ', total_overlap, 
      ' still annotated after the review')
print('This is ', total_overlap/total_chars, '%')
print('We are missing ', whole_row_missing, ' rows where I cannot find the original export')
       

Of  13769  characters orginally annotated  13647  still annotated after the review
This is  0.9911395163047425 %
We are missing  0  rows where I cannot find the original export


In [8]:
#Evaluate How much of the original crowdsourcing annotations were found in the reviewed dataset.   
#This reports based on the span hash ids --> only 100% unchanged spans are counted as same
amt_exp = 0
amt_not_exp = 0

for export_key in export_dict.keys():
    if export_key in review_dict:
        export_comp = export_dict[export_key]['completions']
        review_comp = review_dict[export_key]['completions']
        
        for exp_tag in export_comp[0]['result']:
            contained=False
            if exp_tag['value']['labels'][0] == 'Drug':
            
                for rev_tag in review_comp[0]['result']:
                    if exp_tag['id'] == rev_tag['id']:
                        contained = True

                if contained:
                    amt_exp += 1
                else:
                    amt_not_exp += 1

total_exp_tags = amt_exp + amt_not_exp
print('From a total of ', total_exp_tags, ' tags ', amt_exp, ' remained unchanged after the review')    
print('This is ', amt_exp/total_exp_tags, '%')
        

From a total of  2000  tags  1791  remained unchanged after the review
This is  0.8955 %


In [9]:
#Evaluate How much of the reviewed annotations was already annotated in the crowdsourcing process        
#This reports based on the span hash ids --> only 100% unchanged spans are counted as same
amt_rev = 0
amt_not_rev = 0
whole_row_missing=0

for rev_key in review_dict.keys():
    if rev_key in export_dict:
        export_comp = export_dict[rev_key]['completions']
        review_comp = review_dict[rev_key]['completions']
        
        for rev_tag in review_comp[0]['result']:
            contained=False
            if rev_tag['value']['labels'][0] == 'Drug':
                for exp_tag in export_comp[0]['result']:
                    if exp_tag['id'] == rev_tag['id']:
                        contained = True

                if contained:
                    amt_rev += 1
                else:
                    amt_not_rev += 1
    else:
        whole_row_missing+=1

total_rev_tags = amt_rev + amt_not_rev
print('Of ', total_rev_tags, ' review tags ', amt_rev, ' were already present as is in the export')
print('This is ', amt_rev/total_rev_tags, '%')
print('We are missing ', whole_row_missing, ' rows where I cannot find the original export')


Of  2470  review tags  1791  were already present as is in the export
This is  0.7251012145748987 %
We are missing  138  rows where I cannot find the original export


In [10]:
whole_row_missing

138